In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# --- 1. Define Dataset Paths ---
# Assuming the script is run from a directory where 'face-expression-recognition-dataset' is accessible
dataset_path = '/kaggle/input/face-expression-recognition-dataset/images'
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'validation')

# --- 2. Set Image Parameters ---
IMG_HEIGHT = 224  # Standard input size for many pre-trained models
IMG_WIDTH = 224
BATCH_SIZE = 32

# --- 3. Data Augmentation and Preprocessing ---
# Rescale pixel values to [0, 1] for pre-trained models
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,       # Randomly rotate images by 20 degrees
    width_shift_range=0.2,   # Randomly shift image horizontally
    height_shift_range=0.2,  # Randomly shift image vertically
    shear_range=0.2,         # Apply shearing transformations
    zoom_range=0.2,          # Randomly zoom into images
    horizontal_flip=True,    # Randomly flip images horizontally
    fill_mode='nearest'      # Fill newly created pixels after rotation/shift
)

validation_datagen = ImageDataGenerator(rescale=1./255) # Only rescale for validation

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical' # 'categorical' for multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Get the number of classes from the train generator
num_classes = train_generator.num_classes
print(f"Number of classes detected: {num_classes}")
print(f"Class names: {list(train_generator.class_indices.keys())}")

# --- 4. Load Pre-trained Model (Transfer Learning Base) ---
# Load MobileNetV2 pre-trained on ImageNet, without the top classification layer
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3) # 3 for RGB channels
)

# Freeze the layers of the base model so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# --- 5. Build Custom Classification Head ---
x = base_model.output
x = GlobalAveragePooling2D()(x) # Convert feature maps to a single vector per image
x = Dense(128, activation='relu')(x) # Add a new dense layer
predictions = Dense(num_classes, activation='softmax')(x) # Output layer with 'softmax' for multi-class

# Combine base model and new classification head
model = Model(inputs=base_model.input, outputs=predictions)

# --- 6. Compile the Model ---
model.compile(
    optimizer=Adam(learning_rate=0.0001), # Lower learning rate is often good for transfer learning
    loss='categorical_crossentropy',       # Appropriate for multi-class classification
    metrics=['accuracy']
)

model.summary()

# --- 7. Train the Model ---
EPOCHS = 10 # You might need more epochs for better performance

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE, # Number of batches per epoch
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# --- 8. (Optional) Fine-tuning ---
# After initial training, you can unfreeze some layers of the base model
# and train with a very low learning rate for fine-tuning.
# print("\nStarting fine-tuning...")
# for layer in base_model.layers[-50:]: # Unfreeze the last 50 layers
#     layer.trainable = True
#
# model.compile(
#     optimizer=Adam(learning_rate=0.00001), # Even lower learning rate for fine-tuning
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )
#
# history_fine_tune = model.fit(
#     train_generator,
#     epochs=EPOCHS * 2, # Train for more epochs during fine-tuning
#     initial_epoch=history.epoch[-1],
#     validation_data=validation_generator,
#     steps_per_epoch=train_generator.samples // BATCH_SIZE,
#     validation_steps=validation_generator.samples // BATCH_SIZE
# )

# --- 9. Evaluate the Model (after training/fine-tuning) ---
print("\nEvaluating the model...")
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

# --- 10. Save the Model (optional) ---
model_save_path = 'face_expression_model.h5'
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

2025-07-16 00:00:18.597605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752624018.790331      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752624018.843698      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.
Number of classes detected: 7
Class names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


I0000 00:00:1752624047.474793      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

...

  1/900 ━━━━━━━━━━━━━━━━━━━━ 3:15:34 13s/step - accuracy: 0.2188 - loss: 2.2353

I0000 00:00:1752624064.127020     111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


900/900 ━━━━━━━━━━━━━━━━━━━━ 470s 508ms/step - accuracy: 0.2886 - loss: 1.7699 - val_accuracy: 0.3865 - val_loss: 1.5855
Epoch 2/10
  1/900 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - accuracy: 0.4062 - loss: 1.7601

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


900/900 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.4062 - loss: 1.7601 - val_accuracy: 0.3862 - val_loss: 1.5856
Epoch 3/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 330s 366ms/step - accuracy: 0.3716 - loss: 1.6012 - val_accuracy: 0.4124 - val_loss: 1.5502
Epoch 4/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.4375 - loss: 1.5141 - val_accuracy: 0.4115 - val_loss: 1.5466
Epoch 5/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 342s 380ms/step - accuracy: 0.3968 - loss: 1.5593 - val_accuracy: 0.4314 - val_loss: 1.4918
Epoch 6/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.3750 - loss: 1.6214 - val_accuracy: 0.4341 - val_loss: 1.4937
Epoch 7/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 322s 358ms/step - accuracy: 0.4058 - loss: 1.5375 - val_accuracy: 0.4399 - val_loss: 1.4810
Epoch 8/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.4062 - loss: 1.4486 - val_accuracy: 0.4359 - val_loss: 1.4833
Epoch 9/10
900/900 ━━━━━━━━━━━━━━━━━━━━ 335s 372ms/step - accuracy: 0.4115 - loss: 1.5184 - val_

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# Path ke model yang sudah disimpan
model_save_path = 'face_expression_model.h5'

try:
    loaded_model = tf.keras.models.load_model(model_save_path)
    print(f"Model '{model_save_path}' berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat model: {e}")
    print("Pastikan Anda sudah melatih dan menyimpan model dengan nama 'face_expression_model.h5'.")
    exit() # Keluar jika model tidak bisa dimuat

# Kelas-kelas yang dikenali model Anda (harus sesuai dengan saat pelatihan)
# Anda bisa mendapatkan ini dari train_generator.class_indices.keys()
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Ukuran input yang diharapkan oleh model Anda
IMG_HEIGHT = 224
IMG_WIDTH = 224

Model 'face_expression_model.h5' berhasil dimuat.


In [9]:
def preprocess_and_predict_single_image(model, img_path, target_size, class_names):
    """
    Memuat, memproses, dan memprediksi ekspresi dari satu gambar.

    Args:
        model: Model Keras yang sudah dilatih.
        img_path (str): Path ke file gambar.
        target_size (tuple): Ukuran target (tinggi, lebar) untuk gambar.
        class_names (list): Daftar nama kelas (label) yang dikenali model.

    Returns:
        tuple: (predicted_class_name, confidence)
    """
    try:
        # 1. Muat gambar
        img = image.load_img(img_path, target_size=target_size)
        print(f"Gambar '{img_path}' berhasil dimuat.")

        # 2. Konversi gambar ke array NumPy
        img_array = image.img_to_array(img)

        # 3. Tambahkan dimensi batch (model mengharapkan batch, bahkan untuk satu gambar)
        # Bentuk dari (tinggi, lebar, channels) menjadi (1, tinggi, lebar, channels)
        img_array = np.expand_dims(img_array, axis=0)

        # 4. Normalisasi piksel (sesuai dengan preprocessing saat pelatihan)
        # Jika Anda melakukan rescale=1./255 di ImageDataGenerator, lakukan hal yang sama di sini
        img_array /= 255.0
        print("Gambar berhasil diproses.")

        # 5. Buat prediksi
        predictions = model.predict(img_array)
        # predictions akan menjadi array probabilitas, misal: [[0.01, 0.05, ..., 0.80]]

        # 6. Dapatkan indeks kelas dengan probabilitas tertinggi
        predicted_class_index = np.argmax(predictions[0])

        # 7. Dapatkan nama kelas dan tingkat kepercayaan
        predicted_class_name = class_names[predicted_class_index]
        confidence = predictions[0][predicted_class_index] * 100

        print(f"Prediksi: {predicted_class_name} dengan kepercayaan {confidence:.2f}%")
        return predicted_class_name, confidence

    except FileNotFoundError:
        print(f"Error: File tidak ditemukan di '{img_path}'")
        return None, None
    except Exception as e:
        print(f"Terjadi kesalahan saat memproses gambar: {e}")
        return None, None

In [10]:
# --- Contoh penggunaan ---

# Ganti dengan path ke gambar yang ingin Anda uji
# Misalnya, jika Anda punya gambar di direktori yang sama dengan script Anda:
sample_image_path_1 = '/kaggle/input/hmmmsz/21.jpeg'
sample_image_path_2 = '/kaggle/input/hmmmsz/goyounjung.jpeg'
sample_image_path_3 = '/kaggle/input/hmmmsz/p.jpeg'

print("\n--- Menguji Gambar 1 ---")
preprocess_and_predict_single_image(loaded_model, sample_image_path_1, (IMG_HEIGHT, IMG_WIDTH), class_names)

print("\n--- Menguji Gambar 2 ---")
preprocess_and_predict_single_image(loaded_model, sample_image_path_2, (IMG_HEIGHT, IMG_WIDTH), class_names)

print("\n--- Menguji Gambar 3 ---")
preprocess_and_predict_single_image(loaded_model, sample_image_path_3, (IMG_HEIGHT, IMG_WIDTH), class_names)

# Contoh dengan path yang mungkin tidak ada (untuk demonstrasi error handling)
print("\n--- Menguji Gambar yang Tidak Ada ---")
preprocess_and_predict_single_image(loaded_model, 'gambar_yang_tidak_ada.jpg', (IMG_HEIGHT, IMG_WIDTH), class_names)


--- Menguji Gambar 1 ---
Gambar '/kaggle/input/hmmmsz/21.jpeg' berhasil dimuat.
Gambar berhasil diproses.
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Prediksi: sad dengan kepercayaan 30.09%

--- Menguji Gambar 2 ---
Gambar '/kaggle/input/hmmmsz/goyounjung.jpeg' berhasil dimuat.
Gambar berhasil diproses.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediksi: happy dengan kepercayaan 48.28%

--- Menguji Gambar 3 ---
Gambar '/kaggle/input/hmmmsz/p.jpeg' berhasil dimuat.
Gambar berhasil diproses.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediksi: sad dengan kepercayaan 53.89%

--- Menguji Gambar yang Tidak Ada ---
Error: File tidak ditemukan di 'gambar_yang_tidak_ada.jpg'


(None, None)